# This notebook demos a shallow water stencil

We start with some imports & a small helper function to convert our stencil to SIR:

In [1]:
from typing import Callable

from inspect import getsource
import ast

from dusk.grammar import Grammar

from dawn4py import compile, CodeGenBackend
from dawn4py.serialization import make_sir, SIR
from dawn4py.serialization.SIR import GridType
from dawn4py._dawn4py import run_optimizer_sir


def dusk_to_sir(stencil: Callable) -> SIR:
    # this will give wrong line numbers, there should be a way to fix them
    name = stencil.__name__
    stencil = ast.parse(getsource(stencil))

    assert isinstance(stencil, ast.Module)
    assert len(stencil.body) == 1
    stencil = stencil.body[0]
    assert Grammar.is_stencil(stencil)

    return make_sir(
        name, GridType.Value("Unstructured"), [Grammar().stencil(stencil)]
    )

With the above definitions, we can write our shallow water stencil:

In [2]:
from dusk.script import *


@stencil
def shallow_water(hC: Field[Cell], hC_t: Field[Cell],
                     vC: Field[Cell], vC_t: Field[Cell],
                     uC: Field[Cell], uC_t: Field[Cell],
                     hC_x: Field[Cell], hC_y: Field[Cell], uvC_div: Field[Cell],
                     hE: Field[Edge], vE: Field[Edge], uE: Field[Edge],
                     nx: Field[Edge], ny: Field[Edge], L: Field[Edge], alpha: Field[Edge],
                     boundary_edges: Field[Edge], boundary_cells: Field[Cell],
                     A: Field[Cell], edge_orientation: Field[Cell > Edge],
                     Grav: Field[Cell]):

    with levels_downward:

        # lerp cell quantities to edges
        hE = sum_over(Edge > Cell, hC, weights=[1-alpha, alpha])
        uE = sum_over(Edge > Cell, uC, weights=[1-alpha, alpha])
        vE = sum_over(Edge > Cell, vC, weights=[1-alpha, alpha])
    
        # boundary conditions on cells
        if (boundary_edges):
            uE = 0.
            vE = 0.

        # height field gradient 
        hC_x = sum_over(Cell > Edge, hE * nx * L * edge_orientation) / A
        hC_y = sum_over(Cell > Edge, hE * ny * L * edge_orientation) / A

        # height field gradient is zero on the boundaries
        if (boundary_cells):
            hC_x = 0.
            hC_y = 0.

        # divergence of velocity field
        uvC_div = sum_over(Cell > Edge, (uE*nx + vE*ny)
                           * edge_orientation * L) / A

        # build ODE's
        uC_t = -Grav * hC_x
        vC_t = -Grav * hC_y
        hC_t = hC * uvC_div   


Then we can use dusk to convert the stencil to SIR. With dawn we can compile SIR to C++ which we will write to `shallow_water_cxx-naive.cpp`:

In [3]:
sir = dusk_to_sir(shallow_water)
cpp_naive = compile(sir, backend=CodeGenBackend.CXXNaiveIco)
with open("shallow_water_cxx-naive.cpp", "w+") as f:
    f.write(cpp_naive)

The generated C++ code also requires a driver which is already setup for this demo. With the driver code we can generate an executable `runner`:

In [4]:
!make

First, we put the runner into test mode to ensure that the computed kernel is correct:

In [5]:
!./runner test

mesh stats: #cells 680 #nodes 378 #edges 1057
congratulations!, your stencil is correct, you can visualize now!


If the tester reported that your dusk stencil works correctly, you can now run the complete stencil and visualize. It takes quite a while to run. Its finished after 400 time steps.

In [6]:
!./runner run

mesh stats: #cells 680 #nodes 378 #edges 1057
time 0.04 timestep 2 dt 0.02
time 0.08 timestep 4 dt 0.02
time 0.12 timestep 6 dt 0.02
time 0.16 timestep 8 dt 0.02
time 0.2 timestep 10 dt 0.02
time 0.24 timestep 12 dt 0.02
time 0.28 timestep 14 dt 0.02
time 0.32 timestep 16 dt 0.02
time 0.36 timestep 18 dt 0.02
time 0.4 timestep 20 dt 0.02
time 0.44 timestep 22 dt 0.02
time 0.48 timestep 24 dt 0.02
time 0.52 timestep 26 dt 0.02
time 0.56 timestep 28 dt 0.02
time 0.6 timestep 30 dt 0.02
time 0.64 timestep 32 dt 0.02
time 0.68 timestep 34 dt 0.02
time 0.72 timestep 36 dt 0.02
time 0.76 timestep 38 dt 0.02
time 0.8 timestep 40 dt 0.02
time 0.84 timestep 42 dt 0.02
time 0.88 timestep 44 dt 0.02
time 0.92 timestep 46 dt 0.02
time 0.96 timestep 48 dt 0.02
time 1 timestep 50 dt 0.02
time 1.04 timestep 52 dt 0.02
time 1.08 timestep 54 dt 0.02
time 1.12 timestep 56 dt 0.02
time 1.16 timestep 58 dt 0.02
time 1.2 timestep 60 dt 0.02
time 1.24 timestep 62 dt 0.02
time 1.28 timestep 64 dt 0.02
time 1

In [7]:
%%capture
import prep_animation as pa
import matplotlib.animation as animation
from IPython.display import HTML

animate = animation.FuncAnimation(pa.fig, pa.update_plot, len(pa.out_files), fargs=(pa.zarray, pa.plot), interval = 50)

In [8]:
HTML(animate.to_html5_video())